# Ejercicio en clase: Web Scraping

### Nombre: Hernán Sánchez
### Fecha: 24/01/2025

* Importacion de librerias necesarios como Selenium para automatizar la navegación en Firefox y pandas para procesar los datos extraídos.

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
from datetime import datetime
import pandas as pd
import time
import os
from tqdm import tqdm

* Inicia Firefox con Selenium y abre la página de recetas de AllRecipes.

In [4]:
driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()))

driver.get("https://www.allrecipes.com/recipes-a-z-6735880")

* Configurar Firefox con opciones optimizadas.
* Acceder a la página y maneja reintentos si falla la carga.
* Extraer enlaces de categorías de recetas usando XPath.
* Guardar los datos en un DataFrame y exporta un CSV.
* Cerrar el navegador al finalizar.

In [23]:
try:
    options = webdriver.FirefoxOptions()
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    
    driver = webdriver.Firefox(
        service=Service(GeckoDriverManager().install()),
        options=options
    )
    
    driver.implicitly_wait(10)
    max_retries = 3
    for attempt in range(max_retries):
        try:
            driver.get("https://www.allrecipes.com/recipes-a-z-6735880")
            time.sleep(5)  
            break
        except Exception as e:
            print(f"Intento {attempt + 1} falló: {str(e)}")
            if attempt == max_retries - 1:
                raise
            time.sleep(2)

    element_xpath = '//div[@class="mntl-alphabetical-list__group"]//a'
    
    enlaces_set = set()
    
    max_retries = 3
    for attempt in range(max_retries):
        try:
            wait = WebDriverWait(driver, 20)  
            elements = wait.until(
                EC.presence_of_all_elements_located((By.XPATH, element_xpath))
            )
            break
        except Exception as e:
            print(f"Intento {attempt + 1} de encontrar elementos falló: {str(e)}")
            if attempt == max_retries - 1:
                raise
            time.sleep(2)
    
    for element in elements:
        try:
            article_link = element.get_attribute("href")
            article_text = element.text.strip()
            if article_link and article_text:  
                enlaces_set.add((article_text, article_link))
        except Exception as e:
            print(f"Error al procesar enlace: {str(e)}")
            continue
    
    df = pd.DataFrame(list(enlaces_set), columns=['categoria', 'enlace'])
    
    print("\nPrimeras filas del DataFrame:")
    print(df.head())
    
    print(f"\nTotal de enlaces únicos encontrados: {len(df)}")
    csv_name = f'enlaces_recetas.csv'
    df.to_csv(csv_name, index=False, encoding='utf-8-sig')
    print(f'{csv_name} guardado')

except Exception as e:
    print(f"Error general: {str(e)}")

finally:
    try:
        driver.quit()
    except:
        pass


Primeras filas del DataFrame:
             categoria                                             enlace
0            Spaghetti  https://www.allrecipes.com/recipes/505/main-di...
1            Meatballs  https://www.allrecipes.com/recipes/15455/main-...
2               Gumbos  https://www.allrecipes.com/recipes/1426/soups-...
3  Slow Cooker Recipes  https://www.allrecipes.com/recipes/253/everyda...
4             Cobblers  https://www.allrecipes.com/recipes/361/dessert...

Total de enlaces únicos encontrados: 378
enlaces_recetas.csv guardado


* Ahora ampliamos el Web Scraping para obtener enlaces de recetas dentro de cada categoría.

    - Cargar el CSV con categorías y enlaces.
    - Recorrer cada categoría, accediendo a su URL.
    - Extraer los enlaces de recetas usando XPath.
    - Almacenar los datos en un DataFrame y guarda un CSV con la fecha actual.

In [30]:
df = pd.read_csv('enlaces_recetas.csv')

try:
    options = webdriver.FirefoxOptions()
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    
    driver = webdriver.Firefox(
        service=Service(GeckoDriverManager().install()),
        options=options
    )
    driver.implicitly_wait(10)
    
    resultados = []
    
    for index, row in df.iterrows():
        try:
            print(f"Procesando categoría ({index + 1}/{len(df)}): {row['categoria']}")
            
            driver.get(row['enlace'])
            
            recipe_xpath = '//a[contains(@class, "mntl-card-list-items")]'
            wait = WebDriverWait(driver, 10)
            recipe_elements = wait.until(
                EC.presence_of_all_elements_located((By.XPATH, recipe_xpath))
            )
            
            for recipe in recipe_elements:
                try:
                    recipe_link = recipe.get_attribute('href')
                    if recipe_link:
                        resultados.append({
                            'categoria': row['categoria'],
                            'enlace_categoria': row['enlace'],
                            'enlace_receta': recipe_link
                        })
                except Exception as e:
                    print(f"Error al procesar receta: {str(e)}")
                    continue
                    
        except Exception as e:
            print(f"Error al procesar categoría {row['categoria']}: {str(e)}")
            continue
            
    df_recetas = pd.DataFrame(resultados)
    print("\nPrimeras filas del DataFrame de recetas:")
    print(df_recetas.head())
    print(f"\nTotal de recetas encontradas: {len(df_recetas)}")
    
    fecha_actual = datetime.now().strftime("%Y-%m-%d")
    csv_name = f'enlaces_recetas_detallado_{fecha_actual}.csv'
    df_recetas.to_csv(csv_name, index=False, encoding='utf-8-sig')
    print(f"\nArchivo guardado como: {csv_name}")

except Exception as e:
    print(f"Error general: {str(e)}")

finally:
    try:
        driver.quit()
    except:
        pass

Procesando categoría (1/378): Spaghetti
Procesando categoría (2/378): Meatballs
Procesando categoría (3/378): Gumbos
Procesando categoría (4/378): Slow Cooker Recipes
Procesando categoría (5/378): Cobblers
Procesando categoría (6/378): Frittatas
Procesando categoría (7/378): Leftovers
Procesando categoría (8/378): Filet Mignon
Procesando categoría (9/378): Mediterranean Diet
Procesando categoría (10/378): Chocolate Cakes
Procesando categoría (11/378): Pizza Dough
Procesando categoría (12/378): Cinnamon Rolls
Procesando categoría (13/378): Kolache
Procesando categoría (14/378): Upside-Down Cakes
Procesando categoría (15/378): Pot Pies
Procesando categoría (16/378): Potato Salads
Procesando categoría (17/378): Whoopie Pies
Procesando categoría (18/378): Split Pea Soups
Procesando categoría (19/378): Beef Stroganoff
Procesando categoría (20/378): Turkey
Procesando categoría (21/378): Sugar-Free Recipes
Procesando categoría (22/378): Food Gifts
Procesando categoría (23/378): Chicken and Du

* Finalmente para finalizar la extracción y obtención de los datos mediante el web scrapping, cargamos el CSV con enlaces únicos de recetas.
    - Iniciamos Firefox con Selenium.
    - Recorremos hasta 550 recetas, accediendo a cada enlace.
* Extraemos datos clave:
    - Título y descripción.
    - Tiempos de preparación, cocción y total.
    - Porciones.
    - Lista de ingredientes.
    - Pasos de preparación.
    - Guardamos los datos en CSV con puntos de control cada 50 recetas.

In [ ]:
filename = r"C:\Users\herna\Downloads\ir24b-main\ir24b-main\RI_2024B\data\enlaces_recetas_detallado_2025-01-21.xls"
if not os.path.exists(filename):
   raise FileNotFoundError(f"No se encontró el archivo: {filename}")

df_recetas = pd.read_csv(filename)
enlaces_recetas = df_recetas.drop_duplicates('enlace_receta').head(550)

recetas_count = 0
recetas_procesadas = set()
todas_las_recetas = []

try:
   driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()))
   driver.implicitly_wait(2)

   total_enlaces = len(enlaces_recetas)
   
   with tqdm(total=total_enlaces, desc="Procesando recetas") as pbar:
       for index, row in enlaces_recetas.iterrows():
           if row['enlace_receta'] not in recetas_procesadas:
               try:
                   recetas_procesadas.add(row['enlace_receta'])
                   driver.get(row['enlace_receta'])
                   time.sleep(2)

                   try:
                       titulo = driver.find_element(By.XPATH, '//h1[@class="article-heading text-headline-400"]').text.strip()
                       
                       try:
                           descripcion = driver.find_element(By.XPATH, '//p[@class="article-subheading text-body-100"]').text.strip()
                       except:
                           descripcion = ""

                       prep_time = driver.find_element(By.XPATH, '//div[contains(text(),"Prep Time:")]/following-sibling::div').text.strip()
                       cook_time = driver.find_element(By.XPATH, '//div[contains(text(),"Cook Time:")]/following-sibling::div').text.strip()
                       total_time = driver.find_element(By.XPATH, '//div[contains(text(),"Total Time:")]/following-sibling::div').text.strip()
                       servings = driver.find_element(By.XPATH, '//div[contains(text(),"Servings:")]/following-sibling::div').text.strip()

                       ingredient_elements = driver.find_elements(By.XPATH, '//ul[@class="mm-recipes-structured-ingredients__list"]/li')
                       ingredientes = []
                       for item in ingredient_elements:
                           try:
                               cantidad = item.find_element(By.XPATH, './/span[@data-ingredient-quantity="true"]').text.strip()
                           except:
                               cantidad = ''
                           try:
                               unidad = item.find_element(By.XPATH, './/span[@data-ingredient-unit="true"]').text.strip()
                           except:
                               unidad = ''
                           try:
                               ingrediente = item.find_element(By.XPATH, './/span[@data-ingredient-name="true"]').text.strip()
                           except:
                               ingrediente = ''
                           
                           ingredientes.append(f'{cantidad} {unidad} {ingrediente}'.strip())

                       step_elements = driver.find_elements(By.XPATH, '//ol[@id="mntl-sc-block_1-0"]/li/p[not(contains(@class, "figure-article-caption"))]')
                       pasos = []
                       for i, step in enumerate(step_elements, 1):
                           paso_texto = step.text.strip()
                           if paso_texto:
                               pasos.append(f"Paso {i}: {paso_texto}")

                       receta_info = {
                           'categoria': row['categoria'],
                           'titulo': titulo,
                           'descripcion': descripcion,
                           'tiempo_preparacion': prep_time,
                           'tiempo_cocina': cook_time,
                           'tiempo_total': total_time,
                           'porciones': servings,
                           'ingredientes': '\n'.join(ingredientes),
                           'pasos': '\n'.join(pasos),
                           'url': row['enlace_receta']
                       }
                       
                       todas_las_recetas.append(receta_info)
                       recetas_count += 1
                       pbar.update(1)

                       if recetas_count % 50 == 0:
                           df_temp = pd.DataFrame(todas_las_recetas)
                           fecha_actual = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
                           df_temp.to_csv(f'recetas_allrecipes_checkpoint_{recetas_count}_{fecha_actual}.csv', 
                                        index=False, encoding='utf-8-sig')

                   except Exception as e:
                       print(f"\nError procesando receta {row['enlace_receta']}: {str(e)}")
                       continue

               except Exception as e:
                   print(f"\nError accediendo a URL {row['enlace_receta']}: {str(e)}")
                   continue

except Exception as e:
   print(f"\nError general: {str(e)}")

finally:
   if todas_las_recetas:
       df_final = pd.DataFrame(todas_las_recetas)
       fecha_actual = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
       df_final.to_csv(f'recetas_allrecipes_550_{fecha_actual}.csv', index=False, encoding='utf-8-sig')
   driver.quit()

Comprobamos los datos recopilados

In [5]:
df = pd.read_csv(r'C:\Users\herna\Downloads\ir24b-main\ir24b-main\RI_2024B\notebooks\recetas_allrecipes_checkpoint_550_2025-01-23_22-20-08.csv')
print(df.head())

   categoria                                       titulo  \
0  Spaghetti                Mongolian Ground Beef Noodles   
1  Spaghetti  Creamy Gochujang Spaghetti With Ground Beef   
2  Spaghetti                      Creamy Bacon Pasta Bake   
3  Spaghetti            The Best Million Dollar Spaghetti   
4  Spaghetti           Ham and Butternut Squash Spaghetti   

                                         descripcion tiempo_preparacion  \
0  These Mongolian ground beef noodles have all t...            10 mins   
1  You'll love this creamy gochujang spaghetti wi...            20 mins   
2            "Comforting and a nice weeknight meal."            10 mins   
3  The best million dollar spaghetti is surely th...            20 mins   
4  Despite the rich and decadent mascarpone, the ...            15 mins   

  tiempo_cocina  tiempo_total                           porciones  \
0       25 mins       35 mins                                   4   
1       25 mins       45 mins               

### Pasamos a realizar el buscador de informacion basandonos en el corpus generados con Web Scrapping

* El código preprocesa el texto, convirtiéndolo a minúsculas, eliminando puntuación y stop words, y tokenizándolo para mejorar su análisis posterior.

In [6]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

nltk.download('punkt')
nltk.download('stopwords')

def preprocess_text(text):
    text = text.lower()
    text = ''.join([char for char in text if char not in string.punctuation])
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

df['descripcion'] = df['descripcion'].apply(preprocess_text)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\herna\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\herna\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


* Este código crea una nueva columna corpus en el DataFrame, combinando los textos de las columnas descripcion, ingredientes y pasos de cada receta. Luego, muestra las primeras filas del DataFrame con los títulos y su correspondiente corpus.

In [7]:
df['corpus'] = df['descripcion'] + ' ' + df['ingredientes'] + ' ' + df['pasos']

print(df[['titulo', 'corpus']].head())

                                        titulo  \
0                Mongolian Ground Beef Noodles   
1  Creamy Gochujang Spaghetti With Ground Beef   
2                      Creamy Bacon Pasta Bake   
3            The Best Million Dollar Spaghetti   
4           Ham and Butternut Squash Spaghetti   

                                              corpus  
0  mongolian ground beef noodles flavors love mon...  
1  youll love creamy gochujang spaghetti ground b...  
2                                                NaN  
3  best million dollar spaghetti surely one—a che...  
4  despite rich decadent mascarpone sauce spaghet...  


Para que no se generen errores es importante verificar si hay valores nulos en la columna corpus y eliminar las filas con valores nulos en corpus y reemplaza cualquier valor nulo restante en esa columna por una cadena vacía.

In [12]:
print(df['corpus'].isna().sum())  
df = df.dropna(subset=['corpus'])  
df['corpus'] = df['corpus'].fillna('')  

0


Este código utiliza TfidfVectorizer de scikit-learn para convertir el texto en la columna corpus en una matriz de características numéricas basada en el TF-IDF. Esta transformación mide la importancia de cada palabra en relación con los documentos del corpus. Luego, imprime las dimensiones de la matriz resultante, mostrando el número de documentos y el número de términos únicos.

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

tfidf_matrix = vectorizer.fit_transform(df['corpus'])

print(tfidf_matrix.shape)  

(460, 3640)


* Se procesa la consulta de ingredientes mediante preprocess_text. Luego, convierte la consulta en una representación numérica utilizando TF-IDF y calcula la similitud del coseno entre la consulta y las recetas en el corpus. Selecciona las recetas más similares (las 5 mejores) y las devuelve en un DataFrame con los títulos y los ingredientes de las recetas encontradas.

In [15]:
from sklearn.metrics.pairwise import cosine_similarity

def buscar_recetas_por_ingredientes(query, vectorizer, tfidf_matrix, top_n=5):
    query_procesada = preprocess_text(query)
    
    query_vec = vectorizer.transform([query_procesada])
    
    similitudes = cosine_similarity(query_vec, tfidf_matrix)
    
    indices_similares = similitudes.argsort()[0][-top_n:][::-1] 
    
    return df.iloc[indices_similares]

query_ingredientes = "spaghetti beef garlic soy sauce"
recetas_sugeridas = buscar_recetas_por_ingredientes(query_ingredientes, vectorizer, tfidf_matrix)

print(recetas_sugeridas[['titulo', 'ingredientes']])


                                  titulo  \
0          Mongolian Ground Beef Noodles   
41                 Spaghetti a la Philly   
40            Spaghetti with Corned Beef   
30      Easy Spaghetti with Tomato Sauce   
85  Garlic Ginger Chicken Meatball Bowls   

                                         ingredientes  
0   8 ounces spaghetti\n1 pound ground beef\n4 clo...  
41  1 pound lean ground beef\n1 (24 ounce) jar spa...  
40  8 ounces spaghetti\n1 (12 ounce) can corned be...  
30  1 pound lean ground beef\n2 ½ cups chopped tom...  
85  1  ground chicken\n4 cloves garlic, minced\n1 ...  


### Para mejorar la impresion de los resultados

La función buscar_recetas_por_ingredientes busca las recetas más similares a una consulta de ingredientes utilizando la similitud del coseno entre la consulta procesada y el corpus de recetas. Para cada receta encontrada, resalta los ingredientes que coinciden con la consulta, y devuelve una lista de resultados que incluye detalles de cada receta como título, descripción, tiempos de preparación, ingredientes resaltados y enlace.

In [16]:
import re

def resaltar_coincidencias(ingredientes, query):
    palabras_clave = query.lower().split()
    
    for palabra in palabras_clave:
        ingredientes = re.sub(r'(\b' + re.escape(palabra) + r'\b)', r'<strong>\1</strong>', ingredientes, flags=re.IGNORECASE)
    return ingredientes


In [17]:
def buscar_recetas_por_ingredientes(query, vectorizer, tfidf_matrix, top_n=5):
    query_procesada = preprocess_text(query)
    
    query_vec = vectorizer.transform([query_procesada])
    
    similitudes = cosine_similarity(query_vec, tfidf_matrix)
    
    indices_similares = similitudes.argsort()[0][-top_n:][::-1]  
    
    resultados = []
    for idx in indices_similares:
        receta = df.iloc[idx]
        
        ingredientes_resaltados = resaltar_coincidencias(receta['ingredientes'], query)
        
        resultado = {
            'titulo': receta['titulo'],
            'descripcion': receta['descripcion'],
            'tiempo_preparacion': receta['tiempo_preparacion'],
            'tiempo_cocina': receta['tiempo_cocina'],
            'tiempo_total': receta['tiempo_total'],
            'porciones': receta['porciones'],
            'ingredientes': ingredientes_resaltados,
            'url': receta['url']
        }
        resultados.append(resultado)
    
    return resultados


## Resultados obtenidos por medio de una consulta

In [20]:
query_ingredientes = "chicken"
recetas_sugeridas = buscar_recetas_por_ingredientes(query_ingredientes, vectorizer, tfidf_matrix)

for receta in recetas_sugeridas:
    print(f"**{receta['titulo']}**")
    print(f"Descripción: {receta['descripcion']}")
    print(f"Tiempo de preparación: {receta['tiempo_preparacion']}")
    print(f"Tiempo de cocción: {receta['tiempo_cocina']}")
    print(f"Tiempo total: {receta['tiempo_total']}")
    print(f"Porciones: {receta['porciones']}")
    print(f"Ingredientes: {receta['ingredientes']}")
    print(f"Ver receta completa: {receta['url']}")
    print("\n" + "-"*50 + "\n")


**Slow Cooker Creamy Lemon Herb Chicken**
Descripción: slow cooker creamy lemon herb chicken comes moist tender perfect tossing pasta think creaminess alfredo sauce minus cheese
Tiempo de preparación: 15 mins
Tiempo de cocción: 2 hrs
Tiempo total: 2 hrs 15 mins
Porciones: 6
Ingredientes: 1/2 cup reduced sodium <strong>chicken</strong> broth
2 cloves garlic
3/4 cup heavy cream
1/2 teaspoon Italian seasoning
3/4 teaspoon salt
3 (10 to 12 ounces) <strong>chicken</strong> breasts,
1/2 tespoon freshly ground black pepper
2 tablespoons olive oil,
1 teaspoon lemon zest
2 tablespoons lemon juice
1 tablespoon  cornstarch
(  (
Ver receta completa: https://www.allrecipes.com/slow-cooker-creamy-lemon-herb-chicken-recipe-8711912

--------------------------------------------------

**Incredibly Easy Chicken and Noodles**
Descripción: chicken noodles recipe given aunt known chicken noodles everyone thinks makes scratch recipe easy fast
Tiempo de preparación: 10 mins
Tiempo de cocción: 20 mins
Tiempo 